# LangGraph + Gaugid Store Integration

This notebook demonstrates how to use `GaugidStore` as a `BaseStore` for LangGraph, allowing agents to use Gaugid profiles as persistent key-value stores for checkpoints and state management.

**Installation**: `pip install gaugid[langgraph]`

In [ ]:
import os
import asyncio
from typing import TypedDict, Annotated, Sequence
from operator import add

try:
    from langchain_openai import ChatOpenAI
    from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
    from langgraph.graph import StateGraph, START, END
    from gaugid.integrations.langgraph import GaugidStore
    LANGGRAPH_AVAILABLE = True
except ImportError:
    LANGGRAPH_AVAILABLE = False
    print("⚠️  LangGraph not installed. Install with: pip install gaugid[langgraph]")

# Set your API keys
os.environ["OPENAI_API_KEY"] = "your-key-here"
os.environ["GAUGID_CONNECTION_TOKEN"] = "your-token-here"

In [ ]:
class ChatState(TypedDict):
    """State for the chatbot graph."""
    messages: Annotated[Sequence[BaseMessage], add]
    user_context: str

In [ ]:
# Create Gaugid store
print("1️⃣ Creating GaugidStore for LangGraph...")
store = GaugidStore(
    connection_token=os.getenv("GAUGID_CONNECTION_TOKEN"),
    namespace_prefix=("langgraph", "chatbot"),
    memory_type="episodic",
)
print("   ✅ GaugidStore created")

In [ ]:
# Create LLM
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

# Define chatbot node
def chatbot_node(state: ChatState) -> dict:
    """Main chatbot node that generates responses."""
    system_prompt = f"""You are a helpful AI assistant.

USER CONTEXT (from their Gaugid profile):
{state['user_context']}

Adapt your responses based on the user's preferences and context."""
    
    messages = [SystemMessage(content=system_prompt)] + list(state["messages"])
    response = llm.invoke(messages)
    
    return {"messages": [response]}

# Build graph
print("2️⃣ Creating LangGraph with GaugidStore...")
graph = StateGraph(ChatState)
graph.add_node("chatbot", chatbot_node)
graph.add_edge(START, "chatbot")
graph.add_edge("chatbot", END)

# Compile with GaugidStore as checkpointer
app = graph.compile(checkpointer=store)
print("   ✅ Graph compiled with GaugidStore")

In [ ]:
# Example conversation
config = {"configurable": {"thread_id": "example-thread-1"}}

# Initial state with user context
initial_state = {
    "messages": [HumanMessage(content="Hello! Tell me about yourself.")],
    "user_context": "User is interested in AI and machine learning.",
}

result = app.invoke(initial_state, config)
print(f"User: {initial_state['messages'][0].content}")
print(f"Assistant: {result['messages'][-1].content}")

In [ ]:
# Store some data in the store
print("3️⃣ Storing data in GaugidStore...")
await store.aput(
    namespace=("langgraph", "chatbot", "preferences"),
    key="user_pref_1",
    value={"preference": "prefers technical explanations", "source": "conversation"},
)
print("   ✅ Data stored")

In [ ]:
# Retrieve data
print("4️⃣ Retrieving data from GaugidStore...")
item = await store.aget(
    namespace=("langgraph", "chatbot", "preferences"),
    key="user_pref_1",
)
if item:
    print(f"   ✅ Retrieved: {item.value}")

In [ ]:
# Search data
print("5️⃣ Searching GaugidStore...")
results = await store.asearch(
    namespace_prefix=("langgraph", "chatbot"),
    limit=5,
)
print(f"   ✅ Found {len(results)} items")

In [ ]:
# Cleanup
await store.close()
print("💡 The store persists data in Gaugid profiles across sessions!")